In [1]:
!pip install h5py

ERROR: ld.so: object '/usr/lib/x86_64-linux-gnu/libGLEW.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/usr/lib/x86_64-linux-gnu/libGLEW.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/usr/lib/x86_64-linux-gnu/libGLEW.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/cvmfs/hpc.rug.nl/versions/2023.01/rocky8/x86_64/intel/icelake/software/Python/3.10.4-GCCcore-11.3.0/bin/python -m pip install --upgrade pip' command.


In [2]:
# import NN necessities:
import torch                # tensors etc.
from torch import nn        # nn helpers

# import plotting utilities:
import matplotlib.pyplot as plt

# import data preprocessing utilities:
from sklearn.model_selection import train_test_split
from pathlib import Path    # paths
import h5py                 # for dataset
import numpy as np          # vector calc

# import type annotations:
from typing import List
from typing import Tuple

In [3]:
class Network(nn.Module):   # class defining a basic nn

    def __init__(self, h_size=200, h_layers=4):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(23, h_size),      # in
            nn.ReLU(),
        )
        for i in range(h_layers):
            self.model.append(nn.Linear(h_size, h_size))
            self.model.append(nn.ReLU())
        self.mean_head = nn.Linear(h_size, 18)
        self.logvar_head = nn.Linear(h_size, 18)

        # bind log-variance to avoid numerical instability
        self.max_logvar = nn.Parameter(torch.ones(18) * 0.5)
        self.min_logvar = nn.Parameter(torch.ones(18) * -10)

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)
        self.criterion = nn.MSELoss()    # using mean squared error as a loss metric

    def forward(self, x) -> torch.Tensor:
        res = self.model(x)
        mean = self.mean_head(res)
        logvar = self.logvar_head(res)

        # clamp log-variance using soft constraints (see MBPO/PETS)
        logvar = self.max_logvar - torch.nn.functional.softplus(self.max_logvar - logvar)
        logvar = self.min_logvar + torch.nn.functional.softplus(logvar - self.min_logvar)

        return torch.stack([mean, logvar])

    def nll_loss(self, x, y):
        """
        Negative log-likelihood of Gaussian:
            NLL = 0.5 * [ logσ² + (y - µ)² / σ² ]
        """
        mean, logvar = self.forward(x)
        var = torch.exp(logvar)

        nll = 0.5 * ((y - mean)**2 / var + logvar)
        return nll.mean()

    def train_epoch(self, x, y):
        self.optimizer.zero_grad()

        loss = self.nll_loss(x, y)

        loss.backward()
        self.optimizer.step()
        return loss.item()

    def train(self, x, y, epochs=500, cp=100, surpress=False):
        # again split the data to optimize hyperparam on val set, to not leak data.
        x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, shuffle=True)
    
        test_losses = []
        losses = []
    
        for iter in range(epochs):
            # train
            iteration_loss = self.train_epoch(x_train, y_train)
            losses.append(iteration_loss)

            # validate
            val_loss = self.validation_loss((x_val, y_val))
            test_losses.append(val_loss)
    
            # print
            if not surpress:
                if iter and iter % cp == 0:    # update on iteration checkpoints
                    print(f"iteration {iter}/{epochs}, loss = train: {iteration_loss}, val: {val_loss}")

        return losses, test_losses

    def validation_loss(self, test_data):
        x, y = test_data
        loss = self.nll_loss(x, y)
        return loss.item()

    def reset(self):
        self.__init__()

In [12]:
class Ensemble:

    def __init__(self, n_networks, lmbda=0.5, hidden_size=10, hidden_layers=4):

        self.n_networks = n_networks
        self.models = [
            Network(h_size=hidden_size, h_layers=hidden_layers) for i in range(n_networks)
        ]
        self.lmbda = lmbda

    def train(self, x, y, epochs=500):
        for model in self.models:
            model.train(x, y, epochs=500, surpress=True)

    def test(self, x, y):
        predictions: torch.Tensor = self.predict(x)    # get the prediction
        loss = self.mse(predictions, y)                # calculate the loss
        return loss                                         # report the loss

    def forward(self, x) -> torch.Tensor:
        predictions: torch.Tensor = torch.stack(           # shape: (self.n_models, 2, N, out_dim)
            [model.forward(x) for model in self.models]
        )
        return predictions

    def predict(self, x) -> torch.Tensor:
        predictions: torch.Tensor = self.forward(x)               # forward for all models

        # process ensemble predictions and conclude on 1 prediction
        corrected_prediction: torch.Tensor = self.process_predictions(predictions)

        return corrected_prediction

    def process_predictions(self, predictions: torch.Tensor) -> torch.Tensor:
        # separate means and variances
        means: torch.Tensor = predictions[:, 0, :, :]
        vars: torch.Tensor = predictions[:, 1, :, :]

        # randomly select means
        random_mean = self.select_random_mean(means)

        # calculate max variance
        max_var = self.max_variance(vars)

        # correct reward prediction using max variance
        corrected_prediction = self.penalize_prediction(random_mean, max_var)

        return corrected_prediction

    def select_random_mean(self, means: torch.Tensor):
        # means of shape: (7, N, 18)
        n_nets, n_samples, n_dims = means.shape

        # random network index per sample: (N,)
        idx = torch.randint(
            low=0,
            high=n_nets,
            size=(n_samples, 1),
            device=means.device
        )
        # reshape for gather: (N, 7, 18)
        means_n = torch.permute(means, (1, 0, 2))

        # gather expects index shape to match output shape
        # -> (N, 1, 18), then squeeze to (N, 18)
        idx_g = idx.view(n_samples, 1, 1).expand(-1, 1, n_dims)
        out = means_n.gather(dim=1, index=idx_g).squeeze(1)

        return out
    
    def max_variance(self, vars: torch.Tensor) -> torch.Tensor:
        out = torch.max(vars[:, :, -1], dim=0).values
        return out

    def penalize_prediction(self,
                            mean: torch.Tensor,
                            max_var: torch.Tensor):
        mean[:, -1] = mean[:, -1] - self.lmbda * max_var
        return mean
    
    def mse(self, y: torch.Tensor, y_hat: torch.Tensor):
        mse = (y - y_hat)**2        # mse per prediction
        return torch.mean( mse )    # mean mse

    def to(self, device):
        for model in self.models:                         # move each model to device
            model.to(device)

In [5]:
# load data
data = h5py.File(Path("./halfcheetah_medium-v2.hdf5"))
print([key for key in data.keys()])

['actions', 'infos', 'metadata', 'next_observations', 'observations', 'rewards', 'terminals', 'timeouts']


In [6]:
# extract relevant cols
a = data["actions"]
s_new = data["next_observations"]
s = data["observations"]
r = data["rewards"]

# info
print(
    f"a shape = {a.shape}\n" \
    f"s shape = {s.shape}\n" \
    f"s_new shape = {s_new.shape}\n" \
    f"r shape = {r.shape}\n"
)

a shape = (1000000, 6)
s shape = (1000000, 17)
s_new shape = (1000000, 17)
r shape = (1000000,)



In [7]:
# divide data
x = np.hstack([a, s])                                # -> (N, 23)
y = np.hstack([s_new, np.array(r).reshape(-1, 1)])   # -> (N, 18)

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"using device: {device}")

# converting to tensors
x = torch.tensor(x, dtype=torch.float32).to(device)   
y = torch.tensor(y, dtype=torch.float32).to(device)
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, shuffle=True
)

# info
print(
    f"x_train shape = {x_train.shape}\n" \
    f"x_test shape = {x_test.shape}\n" \
    f"y_train shape = {y_train.shape}\n" \
    f"y_test shape = {y_test.shape}"
)

using device: cpu
x_train shape = torch.Size([800000, 23])
x_test shape = torch.Size([200000, 23])
y_train shape = torch.Size([800000, 18])
y_test shape = torch.Size([200000, 18])


In [ ]:
# train model

# set params
epochs = 100

# for network_width in width_list:
model = Network( h_size=10 )
model = model.to(device)

# train
train_losses, test_losses = model.train(train_data, epochs=epochs, cp=10)
# test_results.append(test_losses)
test_loss = model.nll_loss(x_test, y_test)
print(f"final test loss = {test_loss.item()}")

In [ ]:
# plot data
fig, ax = plt.subplots(1, 1)
x = np.arange(0, epochs)
ax.plot(x, train_losses, label='train', color='red')
ax.plot(x, test_losses, label='test', color='green')
ax.set_xlabel('epochs')
ax.set_ylabel('loss')
plt.legend()
plt.show()


In [11]:
# train ensemble

ensemble = Ensemble(
    n_networks=7, 
    lmbda=0.5,
    hidden_size=10)

ensemble.to(device)

ensemble.train(x_train, y_train, epochs=100)

ensemble.test(x_test, y_test)    # test_data.size: (200_000, 18)



KeyboardInterrupt: 